# Table de synthèse Glottolog

## Objectif

**Production d'un fichier .csv pour les langues d'une famille donnée, à partir de son identifiant Glottolog.**

Pour chaque langue, à partir des données GeoJSON : `ID`, `NOM`, `URI`, `LONGITUDE`, `LATITUDE`.\
Et à partir des données PhyloXML : `HIERARCHIE`.

**Entrée =** Identifiant Glottolog de la famille (type `str` ; 4 lettres, 4 chiffres) stocké dans la variable `lang_id`.\
**Sortie =** Fichier .csv comportant les colonnes `ID`, `NOM`, `URI`, `LONGITUDE`, `LATITUDE` et `HIERARCHIE`.

*Glottolog 4.5 edited by Hammarström, Harald & Forkel, Robert & Haspelmath, Martin & Bank, Sebastian is licensed under a [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/).* 

## Imports

In [1]:
import requests
import csv
from bs4 import BeautifulSoup as soup
from selenium import webdriver
import time
import json

## Fonctions

In [2]:
def get_geojson(langue, driver):
    """Retourne le contenu de la fenêtre modale GeoJSON de la page d'une carte sur Glottolog, parsé avec json.loads()
    Paramètres = langue (str, URL Glottolog), driver (objet webdriver.WebDriver)"""
    map_page = f"{langue}.bigmap.html"
    driver.get(map_page)
    opener = driver.find_element_by_id("legend-geojson-opener")
    opener.click()
    time.sleep(1)
    geo_container = driver.find_element_by_id("legend-geojson-container")
    geo_container.click()
    time.sleep(1)
    page_source = driver.page_source
    page_soup = soup(page_source, 'html.parser')
    modal = page_soup.find(id="ModalBody").text
    return json.loads(modal)

## Variables

Instance de `WebDriver` nécessaire au fonctionnement de `get_geojson()`.

⚠️ Nécessite un répertoire *./WebDrivers/* (`gecko_path`) contenant [geckodriver](https://github.com/mozilla/geckodriver/releases).

In [4]:
gecko_path = "./WebDrivers/"

In [5]:
options = webdriver.FirefoxOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Firefox(gecko_path, options=options)

Identifiant Glottolog pris en entrée (`lang_id`), page Glottolog correspondante et chemin du fichier de sortie.

In [3]:
lang_id = input("ID Glottolog :\n")
lang_page = f"https://glottolog.org/resource/languoid/id/{lang_id}"
output_path = f"{lang_id}.csv"

ID Glottolog :
lolo1267


Récupération et parsing du PhyloXML.

In [6]:
phyloxml_url = f"https://glottolog.org/resource/languoid/id/{lang_id}.phylo.xml"
phyloxml = requests.get(phyloxml_url)
lang_phyloxml = soup(phyloxml.text, features="xml")

Récupération et parsing du GeoJSON via la fonction `get_geojson()`.

In [7]:
lang_geojson = get_geojson(lang_page, driver)

## Programme

Création de la liste `hierarchies` à partir du PhyloXML. Liste de dictionnaires où un dictionnaire = identifiant Glottolog d'une langue (`lid`), situation dans la hiérarchie de la famille de langues (`hierarchie`).

In [8]:
lid_list = []
hierarchies = []
for clade in lang_phyloxml.find_all("clade"):
    lid = clade.find("annotation").find("uri").text.replace("https://glottolog.org/resource/languoid/id/", "")
    hierarchie = clade.find("annotation").find("desc").text
    if lid not in lid_list:
        lid_list.append(lid)
        minihierarchie = {"lid": lid,
                         "hierarchie": hierarchie}
        hierarchies.append(minihierarchie)

Récupération des informations du GeoJSON (nom de la langue, identifiant Glottolog, coordonnées géographiques, URI), association d'une langue avec la hiérarchie correspondante dans la liste `hierarchies`, écriture du fichier de sortie.

In [9]:
with open(output_path, "w", encoding="utf-8", newline="") as cible:
    header = ["NOM", "ID", "URI", "LONGITUDE", "LATITUDE", "HIERARCHIE"]
    writer = csv.DictWriter(cible, delimiter=",", fieldnames=header)
    writer.writeheader()
    for feature in lang_geojson["features"]:
        nom = feature["properties"]["language"]["name"]
        longitude = feature["properties"]["language"]["longitude"]
        latitude = feature["properties"]["language"]["latitude"]
        lid = feature["properties"]["language"]["id"]
        uri = f"https://glottolog.org/resource/languoid/id/{lid}"
        for minihierarchie in hierarchies:
            if minihierarchie["lid"] == lid:
                hierarchie = minihierarchie["hierarchie"]
        minidico = {"NOM": nom,
                   "ID": lid,
                   "URI": uri,
                   "LONGITUDE": longitude,
                   "LATITUDE": latitude,
                   "HIERARCHIE": hierarchie}
        writer.writerow(minidico)

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Licence Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" href="http://purl.org/dc/dcmitype/InteractiveResource" property="dct:title" rel="dct:type">glottolog_synthese - Table de synthèse Glottolog</span> de <a xmlns:cc="http://creativecommons.org/ns#" href="https://tekipaki.hypotheses.org/" property="cc:attributionName" rel="cc:attributionURL">Alexander Delaporte</a> est mis à disposition selon les termes de la <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">licence Creative Commons Attribution -  Partage dans les Mêmes Conditions 4.0 International</a>.<br />Code source disponible : <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/alxdrdelaporte/glottolog_synthese" rel="dct:source">https://github.com/alxdrdelaporte/glottolog_synthese</a>.